# Overview
- Read csv with document data in it
- Split into train/test data
- Train model
- Test model

# Initialization

In [167]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import pdfminer as pdfm

from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

import nltk
nltk.download("punkt")
from nltk.classify import NaiveBayesClassifier

import string
from collections import Counter

[nltk_data] Downloading package punkt to /Users/emilyng/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Read the document data

In [185]:
doc_df = pd.read_csv("Data/doc_list1.csv")
doc_df

,Unnamed: 0,doc_num,doc_filepath,doc_text,text_cleaned,nltk_words,filtered_words,word_counts,bow,env_label
0,0,0,Data/Environmental/PLAW-104publ70.pdf,"PUBLIC LAW 104–70—DEC. 23, 1995\n\n109 STAT. 7...",public law dec stat public law th congressan...,"['public', 'law', 'dec', 'stat', 'public', 'la...","['public', 'law', 'dec', 'stat', 'public', 'la...","[('may', 4), ('occupancy', 4), ('vehicle', 3),...","{'public': 1, 'law': 1, 'dec': 1, 'stat': 1, '...",Environmental
1,1,1,Data/Environmental/PLAW-112publ177.pdf,"PUBLIC LAW 112–177—SEPT. 28, 2012 \n\n126 STAT...",public law sept stat public law th congressa...,"['public', 'law', 'sept', 'stat', 'public', 'l...","['public', 'law', 'sept', 'stat', 'public', 'l...","[('new', 367), ('application', 239), ('use', 2...","{'public': 1, 'law': 1, 'sept': 1, 'stat': 1, ...",Environmental
2,2,2,Data/Environmental/PLAW-116publ63.pdf,"133 STAT. 1120 \n\nPUBLIC LAW 116–63—OCT. 4, 2...",stat public law oct public law th congressan...,"['stat', 'public', 'law', 'oct', 'public', 'la...","['stat', 'public', 'law', 'oct', 'public', 'la...","[('water', 19), ('state', 12), ('revolving', 9...","{'stat': 1, 'public': 1, 'law': 1, 'oct': 1, '...",Environmental
3,3,3,Data/Environmental/PLAW-110publ288.pdf,"PUBLIC LAW 110–288—JULY 29, 2008 \n\nCLEAN BOA...",public law july clean boating act of swalcilb...,"['public', 'law', 'july', 'clean', 'boating', ...","['public', 'law', 'july', 'clean', 'boating', ...","[('management', 14), ('recreational', 12), ('a...","{'public': 1, 'law': 1, 'july': 1, 'clean': 1,...",Environmental
4,4,4,Data/Environmental/PLAW-108publ425.pdf,"PUBLIC LAW 108–425—NOV. 30, 2004\n\nTIJUANA RI...",public law nov tijuana river valley estuary a...,"['public', 'law', 'nov', 'tijuana', 'river', '...","['public', 'law', 'nov', 'tijuana', 'river', '...","[('may', 9), ('act', 7), ('paragraph', 7), ('l...","{'public': 1, 'law': 1, 'nov': 1, 'tijuana': 1...",Environmental
...,...,...,...,...,...,...,...,...,...,...
134,134,134,Data/NonEnvironmental/PLAW-114publ38.pdf,"PUBLIC LAW 114–38—JULY 28, 2015 \n\n129 STAT. ...",public law july stat public law th congressa...,"['public', 'law', 'july', 'stat', 'public', 'l...","['public', 'law', 'july', 'stat', 'public', 'l...","[('business', 21), ('small', 12), ('act', 11),...","{'public': 1, 'law': 1, 'july': 1, 'stat': 1, ...",NonEnvironmental
135,135,135,Data/NonEnvironmental/PLAW-115publ281.pdf,"PUBLIC LAW 115–281—DEC. 1, 2018 \n\n132 STAT. ...",public law dec stat public law th congressan...,"['public', 'law', 'dec', 'stat', 'public', 'la...","['public', 'law', 'dec', 'stat', 'public', 'la...","[('national', 5), ('december', 4), ('act', 4),...","{'public': 1, 'law': 1, 'dec': 1, 'stat': 1, '...",NonEnvironmental
136,136,136,Data/NonEnvironmental/PLAW-115publ280.pdf,"132 STAT. 4190 \n\nPUBLIC LAW 115–280—NOV. 29,...",stat public law nov public law th congressan...,"['stat', 'public', 'law', 'nov', 'public', 'la...","['stat', 'public', 'law', 'nov', 'public', 'la...","[('disaster', 3), ('public', 2), ('law', 2), (...","{'stat': 1, 'public': 1, 'law': 1, 'nov': 1, '...",NonEnvironmental
137,137,137,Data/NonEnvironmental/PLAW-116publ52.pdf,"133 STAT. 1076 \n\nPUBLIC LAW 116–52—AUG. 23, ...",stat public law aug public law th congressan...,"['stat', 'public', 'law', 'aug', 'public', 'la...","['stat', 'public', 'law', 'aug', 'public', 'la...","[('act', 5), ('title', 5), ('aug', 4), ('paid'...","{'stat': 1, 'public': 1, 'law': 1, 'aug': 1, '...",NonEnvironmental


In [186]:
doc_df.shape

(139, 10)

# Split Training vs Testing Data

In [187]:
train_filter = doc_df['doc_num']%2 == 0
test_filter = ~train_filter
#train_filter
#test_filter
training_df = doc_df.loc[train_filter]
testing_df = doc_df.loc[test_filter]

In [188]:
training_df.shape

(70, 10)

In [189]:
testing_df.head()

,Unnamed: 0,doc_num,doc_filepath,doc_text,text_cleaned,nltk_words,filtered_words,word_counts,bow,env_label
1,1,1,Data/Environmental/PLAW-112publ177.pdf,"PUBLIC LAW 112–177—SEPT. 28, 2012 \n\n126 STAT...",public law sept stat public law th congressa...,"['public', 'law', 'sept', 'stat', 'public', 'l...","['public', 'law', 'sept', 'stat', 'public', 'l...","[('new', 367), ('application', 239), ('use', 2...","{'public': 1, 'law': 1, 'sept': 1, 'stat': 1, ...",Environmental
3,3,3,Data/Environmental/PLAW-110publ288.pdf,"PUBLIC LAW 110–288—JULY 29, 2008 \n\nCLEAN BOA...",public law july clean boating act of swalcilb...,"['public', 'law', 'july', 'clean', 'boating', ...","['public', 'law', 'july', 'clean', 'boating', ...","[('management', 14), ('recreational', 12), ('a...","{'public': 1, 'law': 1, 'july': 1, 'clean': 1,...",Environmental
5,5,5,Data/Environmental/PLAW-105publ156.pdf,(cid:80)(cid:85)(cid:66)(cid:76)(cid:73)(cid:6...,cidcidcidcidcidcidcidcidcidcidcidcidcidcidcidc...,['cidcidcidcidcidcidcidcidcidcidcidcidcidcidci...,['cidcidcidcidcidcidcidcidcidcidcidcidcidcidci...,"[('cidcidcidcid', 173), ('cidcidcid', 173), ('...",{'cidcidcidcidcidcidcidcidcidcidcidcidcidcidci...,Environmental
7,7,7,Data/Environmental/PLAW-116publ62.pdf,"133 STAT. 1118 \n\nPUBLIC LAW 116–62—OCT. 4, 2...",stat public law oct oct s alaska remotegene...,"['stat', 'public', 'law', 'oct', 'oct', 's', '...","['stat', 'public', 'law', 'oct', 'oct', 'alask...","[('oct', 5), ('emissions', 4), ('public', 3), ...","{'stat': 1, 'public': 1, 'law': 1, 'oct': 1, '...",Environmental
9,9,9,Data/Environmental/PLAW-111publ191.pdf,"124 STAT. 1278 \n\nPUBLIC LAW 111–191—JUNE 15,...",stat public law june public law th congressa...,"['stat', 'public', 'law', 'june', 'public', 'l...","['stat', 'public', 'law', 'june', 'public', 'l...","[('oil', 7), ('june', 4), ('act', 4), ('advanc...","{'stat': 1, 'public': 1, 'law': 1, 'june': 1, ...",Environmental


In [190]:
training_df.head()

,Unnamed: 0,doc_num,doc_filepath,doc_text,text_cleaned,nltk_words,filtered_words,word_counts,bow,env_label
0,0,0,Data/Environmental/PLAW-104publ70.pdf,"PUBLIC LAW 104–70—DEC. 23, 1995\n\n109 STAT. 7...",public law dec stat public law th congressan...,"['public', 'law', 'dec', 'stat', 'public', 'la...","['public', 'law', 'dec', 'stat', 'public', 'la...","[('may', 4), ('occupancy', 4), ('vehicle', 3),...","{'public': 1, 'law': 1, 'dec': 1, 'stat': 1, '...",Environmental
2,2,2,Data/Environmental/PLAW-116publ63.pdf,"133 STAT. 1120 \n\nPUBLIC LAW 116–63—OCT. 4, 2...",stat public law oct public law th congressan...,"['stat', 'public', 'law', 'oct', 'public', 'la...","['stat', 'public', 'law', 'oct', 'public', 'la...","[('water', 19), ('state', 12), ('revolving', 9...","{'stat': 1, 'public': 1, 'law': 1, 'oct': 1, '...",Environmental
4,4,4,Data/Environmental/PLAW-108publ425.pdf,"PUBLIC LAW 108–425—NOV. 30, 2004\n\nTIJUANA RI...",public law nov tijuana river valley estuary a...,"['public', 'law', 'nov', 'tijuana', 'river', '...","['public', 'law', 'nov', 'tijuana', 'river', '...","[('may', 9), ('act', 7), ('paragraph', 7), ('l...","{'public': 1, 'law': 1, 'nov': 1, 'tijuana': 1...",Environmental
6,6,6,Data/Environmental/PLAW-114publ182.pdf,"PUBLIC LAW 114–182—JUNE 22, 2016 \n\nFRANK R. ...",public law june frank r lautenberg chemical s...,"['public', 'law', 'june', 'frank', 'r', 'laute...","['public', 'law', 'june', 'frank', 'lautenberg...","[('administrator', 295), ('chemical', 253), ('...","{'public': 1, 'law': 1, 'june': 1, 'frank': 1,...",Environmental
8,8,8,Data/Environmental/PLAW-111publ378.pdf,"124 STAT. 4128 \n\nPUBLIC LAW 111–378—JAN. 4, ...",stat public law jan public law th congressan...,"['stat', 'public', 'law', 'jan', 'public', 'la...","['stat', 'public', 'law', 'jan', 'public', 'la...","[('stormwater', 6), ('federal', 5), ('jan', 4)...","{'stat': 1, 'public': 1, 'law': 1, 'jan': 1, '...",Environmental


# Train the Model

In [10]:
def create_nb_input(doc_df, label) :
    ctr = 0
    for word_list in doc_df['bow'].tolist() :
#        print(type(word_list))
#        print(word_list)
#        break
        if ctr == 0 :
            nb_input = [(word_list, label)]
        else :
            nb_input.append((word_list, label))
        ctr += 1
    return nb_input

In [11]:
filter = training_df['env_label'] == "Environmental"
env_training_df = training_df.loc[filter]
notenv_training_df = training_df.loc[~filter]

In [12]:
nb_env_input = create_nb_input(env_training_df, 'Environmental')
nb_notenv_input = create_nb_input(notenv_training_df, 'NotEnvironmental')
print(nb_env_input[0][0])

{'public': 1, 'law': 1, '112–177—sept': 1, 'stat': 1, '1327public': 1, '112–177112th': 1, 'congressan': 1, 'actto': 1, 'reauthorize': 1, 'federal': 1, 'insecticide': 1, 'fungicide': 1, 'rodenticide': 1, 'act.be': 1, 'enacted': 1, 'senate': 1, 'house': 1, 'representatives': 1, 'ofthe': 1, 'united': 1, 'states': 1, 'america': 1, 'congress': 1, 'assembled': 1, 'section': 1, '1.': 1, 'short': 1, 'title.this': 1, 'act': 1, 'may': 1, 'cited': 1, 'pesticide': 1, 'registration': 1, 'improve-ment': 1, 'extension': 1, '.sept': 1, 's.': 1, 'pesticideregistrationimprovementextension': 1, 'actof': 1, '2012.7': 1, 'usc': 1, 'note.sec': 1, '2.': 1, 'improvement': 1, 'maintenance': 1, 'fees.—': 1, 'fees.—section': 1, 'u.s.c': 1, '136a–1': 1, 'amended—': 1, 'paragraph': 1, 'subparagraph': 1, 'striking': 1, 'aggregateamount': 1, 'follows': 1, 'end': 1, 'inserting': 1, 'aggregate': 1, 'amount': 1, '27,800,000': 1, 'fiscal': 1, 'years': 1, 'through2017.': 1, 'ii': 1, 'clause': 1, 'shall': 1, 'andall': 1, 

In [13]:
print(type(nb_env_input))
print(len(nb_env_input))

<class 'list'>
12


In [14]:
legislative_subject_classifier = NaiveBayesClassifier.train(nb_env_input[0][0] + nb_notenv_input[0][0])
#legislative_subject_classifier = NaiveBayesClassifier.train(nb_env_input[0][0] + nb_notenv_input[0][0])
#legislative_subject_classifier = NaiveBayesClassifier.train(nb_env_input[0][0] + nb_notenv_input[0][0])

ValueError: not enough values to unpack (expected 2, got 1)

In [ ]:
nltk.classify.util.accuracy(mr_op_classifier, nb_mr_train_input + nb_op_train_input)*100

# Test the model
- essentially just run the accuracy but pass in the testing data instead of the training data

### Using TFIDF vectorizer

In [467]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score, roc_auc_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

def vectorize(X_text_tr, X_text_val, vectorizer_type):
    '''
    Vectorizes text into machine readable and model ready input format.
    
    Input:
        X_text_tr (Training set texts)
        X_text_val (Validation set texts)
        vectorizer_type ['tfidf', 'counter']
    Output:
        X_tr_vector: sparse matrix; vectorized training text features
        X_val_vector: sparse matrix; vectorized validation text features
        train_df: doc-term term frequencies dataframe (training)
        val_df: 
        vectorizer: vectorizer object; fitted on X_text_tr to be used to transform test text
    '''
    if vectorizer_type == 'tfidf':
        vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.1, lowercase=True, analyzer='word',
                                 stop_words= 'english',ngram_range=(1,1))
    elif vectorizer_type == 'counter':
        vectorizer = TfidfVectorizer(max_df=0.8, min_df=0.1, lowercase=True, analyzer='word',
                                 stop_words= 'english',ngram_range=(1,1))
        
    X_tr_vector = vectorizer.fit_transform(X_text_tr.ravel())
    X_val_vector = vectorizer.transform(X_text_val.ravel())
    
    feature_names = vectorizer.get_feature_names()
    dense1 = X_tr_vector.todense().tolist()
    dense2 = X_val_vector.todense().tolist()
    train_df = pd.DataFrame(dense1, columns=feature_names)
    val_df = pd.DataFrame(dense2, columns=feature_names)
    
    return X_tr_vector, X_val_vector, train_df, val_df, vectorizer

In [457]:
text = doc_df['text_cleaned']
y = (doc_df['env_label'] == 'Environmental')

#test random_state
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.2)
text_train, text_val, y_train, y_val = train_test_split(text_train, y_train, test_size=0.2)

kf = StratifiedKFold(n_splits=5, shuffle=True)
#model using TFIDF vectorizer
X_train, X_val, df_train, df_val, vectorizer = vectorize(text_train, text_val, 'tfidf')
X, y = X_train.toarray(), np.array(y_train)

train_accuracies, train_precisions, train_recalls, train_f1s, train_fbetas = [], [], [], [], []
val_accuracies, val_precisions, val_recalls, val_f1s, val_fbetas = [], [], [], [], []

for train_ind, val_ind in kf.split(X,y):
    X_train, y_train = X[train_ind], y[train_ind]
    X_val, y_val = X[val_ind], y[val_ind] 
    
    model = BernoulliNB()
    model.fit(X_train, y_train)
    ytrain_preds = model.predict(X_train)
    ytrain_preds_probs = model.predict_proba(X_train)[:,1]
    train_preds = np.where(ytrain_preds_probs > 0.5, 1, 0)
    yval_preds = model.predict(X_val)
    yval_preds_probs = model.predict_proba(X_val)[:,1]
    val_trains = np.where(yval_preds_probs > 0.5, 1, 0)
    
#     train_acc = model.score(X_train, y_train)
#     val_acc = model.score(X_val, y_val)
    train_accuracies.append(accuracy_score(y_train, ytrain_preds))
    train_precisions.append(precision_score(y_train, train_preds))
    train_recalls.append(recall_score(y_train, ytrain_preds))
    train_f1s.append(f1_score(y_train, ytrain_preds))   
    train_fbetas.append(fbeta_score(y_train, ytrain_preds, beta=0))

    val_accuracies.append(accuracy_score(y_val, yval_preds))
    val_precisions.append(precision_score(y_val, yval_preds))
    val_recalls.append(recall_score(y_val, yval_preds))
    val_f1s.append(f1_score(y_val, yval_preds))   
    val_fbetas.append(fbeta_score(y_val, yval_preds, beta=0))
    
train_scores = [train_accuracies, train_precisions, train_recalls, train_f1s, train_fbetas]
val_scores = [val_accuracies, val_precisions, val_recalls, val_f1s, val_fbetas]
scores = ['Accuracy', 'Precision', 'Recall', 'F1', 'FBeta']

In [459]:
for score, tr, val in zip(scores, train_scores, val_scores):
    print(score)
    print(f'Train: {np.mean(tr):.3f} +- {np.std(tr):.3f}')
    print(f'Val: {np.mean(val):.3f} +- {np.std(val):.3f}')
    print('-------------')

Accuracy
Train: 0.940 +- 0.021
Val: 0.795 +- 0.059
-------------
Precision
Train: 0.945 +- 0.033
Val: 0.787 +- 0.130
-------------
Recall
Train: 0.912 +- 0.041
Val: 0.754 +- 0.110
-------------
F1
Train: 0.928 +- 0.026
Val: 0.756 +- 0.061
-------------
FBeta
Train: 0.945 +- 0.033
Val: 0.787 +- 0.130
-------------


### Testing Model on Hold-out/Test Set

In [466]:
X_test = vectorizer.transform(text_test)
preds = model.predict(X_test)
print(f'Test Accuracy: ', model.score(X_test, y_test))

Test Accuracy:  0.9285714285714286


In [464]:
#save model in pickle format
pd.to_pickle(model, 'env_classifier.pkl')

### Extras

In [262]:
#No Kfold cross validation yet
def run_model_n_times(n_times):
    train_scores, test_scores = [], []
    
    runs = 0
    while runs < n_times:
        print(f'{runs+1} Run(s)')
        text_train, text_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
        #first model using TFIDF vectorizer
        X_train, X_test, df_train, df_test, vectorizer = vectorize(text_train, text_test, 'tfidf')
        X_train.shape, X_test.shape

        model = BernoulliNB()
        model.fit(X_train, y_train)
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)
        
        train_scores.append(train_score)
        test_scores.append(test_score)
        print('Train Score:', train_score)
        print('Test Score:', test_score)
        print('--------------------------')
        runs+=1
        
    print(f'Average Train Score ({n_times} Runs): {np.mean(train_scores)}')
    print(f'Average Test Score ({n_times} Runs): {np.mean(test_scores)}')

run_model_n_times(10)

1 Run(s)
Train Score: 0.8557692307692307
Test Score: 0.7714285714285715
--------------------------
2 Run(s)
Train Score: 0.8846153846153846
Test Score: 0.8
--------------------------
3 Run(s)
Train Score: 0.875
Test Score: 0.9428571428571428
--------------------------
4 Run(s)
Train Score: 0.75
Test Score: 0.6285714285714286
--------------------------
5 Run(s)
Train Score: 0.9326923076923077
Test Score: 0.9142857142857143
--------------------------
6 Run(s)
Train Score: 0.75
Test Score: 0.7142857142857143
--------------------------
7 Run(s)
Train Score: 0.8076923076923077
Test Score: 0.7428571428571429
--------------------------
8 Run(s)
Train Score: 0.7403846153846154
Test Score: 0.5714285714285714
--------------------------
9 Run(s)
Train Score: 0.8557692307692307
Test Score: 0.7428571428571429
--------------------------
10 Run(s)
Train Score: 0.8653846153846154
Test Score: 0.8857142857142857
--------------------------
Average Train Score (10 Runs): 0.8317307692307694
Average Test Sc

### Using CounterVectorizer

In [226]:
X_train, X_test, df_train, df_test, vectorizer = vectorize(text_train, text_test, 'counter')
X_train.shape, X_test.shape

((104, 475), (35, 475))

In [227]:
model = BernoulliNB()
model.fit(X_train, y_train)
train_score = model.score(X_train, y_train)
test_score = model.score(X_test, y_test)
print('Train Score:', train_score)
print('Test Score:', test_score)

Train Score: 0.8173076923076923
Test Score: 0.7142857142857143


After trying out both TFIDF and Counter vectorizers, neither really have any discernable difference in performance.